# Using timestamps in MISP

## Introduction

- UUID: **7b498d8f-d1a3-4576-b894-58f908fff82c**
- Started from [issue 42](https://github.com/MISP/misp-playbooks/issues/42)
- State: **Published**
- Purpose:
    - A playbook that documents the different timestamps that are used in MISP.
    - Go through the timestamp for publishing and last changes as well as how these can be used in search queries.
    - Document what changes a timestamp in a MISP event.
- Tags: [ "timestamp", "export" ]
- External resources: None
- Target audience: **CTI**

# Playbook TOC

- **Using timestamps in MISP**
   - Introduction
- **Timestamps in MISP**
   - IN:1 Introduction
   - IN:2 Date formats
- **Preparation**
   - PR:1 Initialise environment
   - PR:2 Load helper functions
- **Adding events and attributes to MISP**
   - AD:1 Create an event
   - AD:2 Add attributes to an event
   - AD:3 Publish the event
   - AD:4 Summary
- **Event search context**
   - ES:1 Search for recently published events
   - ES:2 Search for recently changed events
   - ES:3 Search for events on their date field
   - ES:4 What is the "First recorded change" of an event?
- **Attribute search context**
   - AS:1 Search for attributes on their last change timestamp
   - AS:2 Search for attributes on their first seen or last seen value
   - AS:3 Search for attributes in events with a specific recent change or published timestamp
   - AS:4 Search for attributes in events with a specific date field
   - AS:5 Search for first change of an attribute
- **Object search context**
   - OS:1 Search for objects
- **Sightings**
   - SI:1 Adding sightings
   - SI:2 Search for sightings
- **Proposal / ShadowAttributes**
   - PS1: Search for proposals or shadowattributes
- **Closure**
   - EN:1 End of the playbook
   - External references
   - Technical details

# Timestamps in MISP

## IN:1 Introduction

All the activities in MISP, such as creating events, adding attributes or editing objects, are timestamped. These timestamps are not only tracked in the audit log but are an integral part of the [MISP data model](https://www.misp-project.org/misp-training/a.11-misp-data-model.pdf) for events, attributes or objects.

## IN:2 Date formats 

Before we continue it's useful to review the different date formats that you can use (primarily for searching) in MISP. You can use 

- An exact **date**, such as *2023-07-10*, sometimes with and sometimes without hour and minute representation. Corresponds with ISO 8601 date notation.
- An exact **UNIX timestamp** match, such as *1688655953*
- A **short-hand** notation, such as *1d* or *3h*
- A **date range**, such as *["14d", "7d"]*

# Preparation

## PR:1 Initialise environment

This section **initialises the playbook environment** and loads the required Python libraries. The credentials for MISP (**API key**) are loaded from the file `keys.py` in the directory **vault**. A [PyMISP](https://github.com/MISP/PyMISP) object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

In [ ]:
# Initialise Python environment
import urllib3
import sys
import json
from datetime import date
import datetime
from prettytable import PrettyTable, MARKDOWN
from pymisp import *
import requests

# Load the credentials
sys.path.insert(0, "../vault/")
from keys import *
if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
print("I will use the MISP server \033[92m{}\033[90m for this playbook.\n\n".format(misp_url))

# Set the headers in case we use Python requests
headers = {"Authorization": misp_key,
           "Accept": "application/json",
           "Content-type": "application/json"}

## PR:2 Load helper functions

The next cell contains **helper functions** that are used in this playbook. 

In [ ]:
# Return a PrettyTable object with columns 'fields'
def eventstable(fields):
    table = PrettyTable()
    table.field_names = fields
    table.align["Event"] = "l"
    table.align["Event ID"] = "l"
    table._max_width = {"Event": 45}
    return table


# Convert a UNIX timestamp to a human readable date format 
def unix_timestamp_to_date(timestamp, display_format="%Y-%m-%d %H:%M:%S"):
    timestamp = int(timestamp)
    dt = datetime.datetime.fromtimestamp(timestamp)
    return dt.strftime(display_format)

# Adding events and attributes to MISP

## AD:1 Create an event

When you **add a MISP event** you need to provide a date field. This date refers to when a threat was observed or when an incident occurred. The [date](https://www.misp-standard.org/rfc/misp-standard-core.html#name-date) field is in ISO 8601 format (**YYYY-MM-DD**). When you manually add an event via the MISP user interface you can use a date picker to set the correct date. 

This date field is set by the person (or script) that created the threat event, this timestamp is not set automatically by MISP.

![helpers/timestamp_add.png](helpers/timestamp_add.png)

### Use PyMISP to create an event

The example uses PyMISP to create a simple MISP event, sets a date with **set_date** and then requests the event from the MISP server.

In [ ]:
event = MISPEvent()
event.info = "Demo event for MISP"
event.distribution = 3
event.threat_level_id = 4
event.analysis = 1

# Set the MISP event date
event.set_date("2023-07-23")

result = misp.add_event(event)["Event"]
event_uuid = result["uuid"]
timestamp_on_create = result["timestamp"]

print("Created event ID \033[92m{}\033[90m with UUID \033[92m{}\033[90m.\n\n{}\n".format(result["id"], result["uuid"], result))
print("Date ('date'): \033[92m", result["date"], "\033[90m")
print("Publish timestamp ('publish_timestamp'): \033[92m", result["publish_timestamp"], "\033[90m")
print("Timestamp ('timestamp'): \033[92m", timestamp_on_create, "\033[90m")

### Output from PyMISP after creating the event

As you can notice from the previous output:
- The **date** is the value supplied during event creation. It is in ISO 8601 format.
- The **publish_timestamp** is not yet set. It is set by MISP following a publishing action and is covered in a later section.
- The **timestamp** indicates the last change of the event, is set automatically by MISP and is returned in UNIX timestamp format.

## AD:2 Add attributes to an event

Next we add an attribute to the event. There are two *optional* dates that you can supply:

- The **first seen**, indicating when you have first seen the attribute
- The **last seen**, when you have last seen the attribute

These dates add contextualisation, in the sense that they allow you to indicate if a threat (an indicator) is still relevant. They are *optional*, you can combine them or only add the first seen, and not the last seen, or vice-versa. The [first seen and last seen](https://www.misp-standard.org/rfc/misp-standard-core.html#name-first_seen) are in an ISO 8601 format, up to the micro-second with time zone support.

![helpers/timestamp_add_attribute.png](helpers/timestamp_add_attribute.png)

When you add attributes with the *freetext import tool*, you do not get the option to add the first seen or last seen. You can still add them later on, once the attributes are added to the event. MISP also does not show the first seen or last seen value when you list the attributes in a threat event. To view them you have to click on the **Context** button. You can also view them in a more visual representation under the **Event timeline**.

![helpers/timestamp_attribute_seen.png](helpers/timestamp_attribute_seen.png)

### Use PyMISP to add an attribute 

With PyMISP you can provide the first seen (**first_seen**) and last seen (**last_seen**) value in UNIX timestamp format but also as a human readable date and time.

There's one additional timestamp in the example below and that is the attribute **timestamp**. You can specify a timestamp when you create an attribute, but MISP will ignore this value. The timestamp is set **automatically** by MISP to the time when the action is executed, you cannot override this.

In [ ]:
attribute = MISPAttribute()
attribute.type = "ip-dst"
attribute.value = "1.2.2.1"
attribute.to_ids = True
attribute.comment = "There is an initial request to this IP"

attribute.timestamp = 8551954799 # This will be ignored
attribute.first_seen = 1689020494  # Provide first seen in UNIX timestamp format
attribute.last_seen = "2023-07-15" # Provide last seen in human readable date format

result_attr = misp.add_attribute(event_uuid, attribute)["Attribute"]
result_attr_uuid = result_attr["uuid"]
attribute_timestamp = result_attr["timestamp"]

print("Added attribute \033[92m{}\033[90m\n\n{}\n".format(result_attr_uuid, result_attr))
print("Timestamp ('timestamp'): \033[92m", attribute_timestamp, "\033[90m")
print("First seen ('first_seen'): \033[92m", result_attr["first_seen"], "\033[90m")
print("Last seen ('last_seen'): \033[92m", result_attr["last_seen"], "\033[90m")

## AD:3 Publish the event

When an event is created it is visible on the MISP instance (depending on the event distribution setting) but it is not exportable or synchronised with other servers. 

![helpers/timestamp_publishing.png](helpers/timestamp_publishing.png)

It's only after **publishing the event** that it is exported or synchronised. When you publish an event its publish timestamp is **automatically** set by MISP. 

### Use PyMISP to publish an event

Publishing a MISP event with PyMISP is easy with the **publish** function.

In [ ]:
print("Publish MISP event \033[92m{}\033[90m".format(event_uuid))
print(misp.publish(event_uuid))

### Use PyMISP to retrieve details of the event

Next we query MISP for the event that we just published and analyse which timestamps have changed.

In [ ]:
result = misp.search(controller="events", uuid=event_uuid)[0]["Event"]
print(result, "\n")

print("Date ('date'): \033[92m", result["date"], "\033[90m")
print("Publish timestamp ('publish_timestamp'): \033[92m", result["publish_timestamp"], "\033[90m")
print("Timestamp ('timestamp'): \033[92m", result["timestamp"], "\033[90m")
print("Timestamp when we created the event ('timestamp'): \033[92m", timestamp_on_create, "\033[90m")
if result["timestamp"] == attribute_timestamp:
    print("Timestamp when we added the attribute ('timestamp'): \033[92m", attribute_timestamp, "\033[90m")
else:
    print("There have been other changes after adding attributes in this playbook.\n   For reference, the timestamp when we added the attribute ('timestamp'): \033[92m", attribute_timestamp, "\033[90m")

### Results of adding data with PyMISP

As you can observe in the above example:

- The **publish timestamp** is updated because of the publish action. This timestamp is set by MISP and returned in UNIX timestamp format.
- The last change (**timestamp**) of the event corresponds with the time the attribute was added. This timestamp is also set by MISP and returned in UNIX timestamp format.

## AD:4 Summary

In the previous cells we breefly covered how to add events and attributes and which timestamps come into play. A lot of **use cases for timestamps** in MISP deal with searching (and filtering) for data. This is covered in the next section.

# Event search context

## ES:1 Search for recently published events

When you publish an event it's **publish timestamp** is automatically updated by MISP. Searching for events based on their publish timestamps then allows you to filter for recently created (or synchronised) events made available on your MISP server. The [publish_timestamp](https://www.misp-standard.org/rfc/misp-standard-core.html#name-publish_timestamp) is expressed in seconds (decimal) since 1st of January 1970 (Unix timestamp). At each publication of an event, the publish_timestamp is. If the published_timestamp is present and the published flag is set to false, the publish_timestamp represents the previous publication timestamp. If the event was never published, the published_timestamp is set to 0.

This publish timestamp corresponds with what's on the *Published* line when you view the details of a MISP event. It is set on *your* MISP instance, when you published the MISP event on your instance (either automatically for example after syncing or manually because of a follow-up action).

![helpers/timestamp_published.png](helpers/timestamp_published.png)

### Search for published or unpublished events

The published timestamp represents the timestamp when the event was **last published**. It is not an indication if the event is **currently published**. To clarify: an event can be in an unpublished state, and still have a published timestamp, referring to a previous action. If you do not specify to PyMISP to only return published or unpublished events, then it will return **both**, the published and unpublished events.

### Use PyMISP to search for events with a recent publish timestamp and use a short-hand date format

In the PyMISP search function set the controller to **events** (the default setting) and use the **publish_timestamp** parameter (technically you can also use the **last** parameter, it is a synonym for publish_timestamp but is deprecated). 

In [ ]:
# Get events with a publish timestamp of 10d or less
print("Search for \033[92mevents with a recent publish timestamp\033[90m and use a \033[92mshort-hand\033[90m date format")
events = misp.search("events", publish_timestamp="10d", pythonify=True)
table = eventstable(["Event ID", "Event", "Published date", "Published"])
for event in events:
    table.add_row([event.id, event.info, event.publish_timestamp, event.published])
print(table.get_string(sortby="Published date"), "\n\n")

In [ ]:
# Get published events with a publish timestamp of 10d or less
print("Search for \033[92mevents with a recent publish timestamp\033[90m, use a \033[92mshort-hand\033[90m date format and only return \033[92mpublished\033[90m events.")
events = misp.search("events", publish_timestamp="10d", pythonify=True, published=1)
table = eventstable(["Event ID", "Event", "Published date", "Published"])
for event in events:
    table.add_row([event.id, event.info, event.publish_timestamp, event.published])
print(table.get_string(sortby="Published date"), "\n\n")

In [ ]:
# Get the not-published events with a publish timestamp of 10d or less
print("Search for \033[92mevents with a recent publish timestamp\033[90m, use a \033[92mshort-hand date\033[90m format and only return \033[92mnot-published\033[90m events.")
events = misp.search("events", publish_timestamp="10d", pythonify=True, published=0)
table = eventstable(["Event ID", "Event", "Published date", "Published"])
for event in events:
    table.add_row([event.id, event.info, event.publish_timestamp, event.published])
print(table.get_string(sortby="Published date"), "\n\n")

### Use PyMISP to search for published events during a specific date range specified in short-hand format

Instead of specifying a date in short-hand format you can also specify a **date range** with the format *["startdate","enddate"]*. The below example searches for events published between 10 and 3 days ago.

In [ ]:
print("Search for \033[92mevents with a recent publish timestamp\033[90m, use a date \033[92mrange\033[90m and only return \033[92mpublished\033[90m events.")
events = misp.search("events", publish_timestamp=["3d","10d"], pythonify=True, published=1)
table = eventstable(["Event ID", "Event", "Published date", "Published"])
for event in events:
    table.add_row([event.id, event.info, event.publish_timestamp, event.published])
print(table.get_string(sortby="Published date"), "\n\n")

### Use REST API to search for published events on an exact date

The REST API uses a similar syntax as PyMISP. The difference is that instead of providing the parameters to PyMISP, you now have to supply them in JSON format: `{"returnFormat":"json", "publish_timestamp": "yourdate", "published":"1"}`. The result shows one of the advantages of using PyMISP. If you use the REST API you get the timestamps in UNIX timestamp, which you then need to convert to a human readable format. With PyMISP a date object is returned, provided you use the **pythonify** setting.

In [ ]:
print("Search for \033[92mpublished events on an exact date\033[90m")
result = requests.post("{}/events/restSearch".format(misp_url), headers=headers, verify=misp_verifycert,
                           json={"returnFormat":"json", "publish_timestamp":"2023-07-07", "published":"1"})
table = eventstable(["Event ID", "Event", "Published date", "Published Converted", "Published"])
if not "errors" in result.json():    
    for event in result.json().get("response", []):
        table.add_row([event["Event"]["id"], event["Event"]["info"], event["Event"]["publish_timestamp"], unix_timestamp_to_date(event["Event"]["publish_timestamp"]), event["Event"]["published"]])
print(table.get_string(sortby="Published date"), "\n\n")

## ES:2 Search for recently changed events

Searching for recently changed events is with the **timestamp** parameter. Similarly as with searching for the publish timestamp, you can specify a short-hand notation, an exact timestamp or a range. If you do not supply the published state as a parameter then both published and unpublished events are returned. The [timestamp](https://www.misp-standard.org/rfc/misp-standard-core.html#name-timestamp) represents a reference time when the event, or one of the attributes within the event was created, or last updated/edited on the instance. Timestamp is expressed in seconds (decimal) since 1st of January 1970 (Unix timestamp).

This timestamp corresponds with the *Last change* line when you view the details of MISP event. This can be a timestamp set on *your* MISP instance (if you did a change to an event) but can also be set on a remote MISP server when you synchronise threat data.

![helpers/timestamp_lastchange.png](helpers/timestamp_lastchange.png)

### Use PyMISP to search for recently changed events

Searching for recently changed events in PyMISP is similar as searching for events based on their publish_timestamp. Set the controller to events and use the **timestamp** parameter.

In [ ]:
print("Search for \033[92mrecently changed\033[90m events with \033[92mshort-hand\033[90m format")
events = misp.search("events", timestamp="3d", pythonify=True)
table = eventstable(["Event ID", "Event", "Last change", "Published"])
for event in events:
    table.add_row([event.id, event.info, event.timestamp, event.published])
print(table.get_string(sortby="Last change"), "\n\n")

### Use REST API to search for recently changed events

Use the `timestamp` field in the JSON request that you send to the REST API: `{"returnFormat":"json", "timestamp": "yourdate"}`.

## ES:3 Search for events on their date field

A MISP event has a date field that represents when a threat was observed or when an incident occured. Whereas the previous searches are done on the fields that are **automatically** set by MISP because of user actions, this search is done on a field **manually** set by a MISP user.

This timestamp corresponds with the *Date* line when you view the details of MISP event and is set by the creator of the threat event.

![helpers/timestamp_date.png](helpers/timestamp_date.png)

### Use PyMISP to search for events of specific dates

You can search for events with a date set after the one specified with **date_from** or for events with a date set before the one specified with **date_to**. It's possible to combine both settings, allowing you to specify a date range.

In [ ]:
print("Search for \033[92mevents after and before a specific date\033[90m")
events = misp.search("events", date_from="2023-07-04", date_to="2023-07-06", pythonify=True)
table = eventstable(["Event ID", "Event", "Date", "Published"])
for event in events:
    table.add_row([event.id, event.info, event.date, event.published])
print(table.get_string(sortby="Date"), "\n\n")   

### Use REST API to search for events of specific dates

The previous search queries for PyMISP have a similar syntax when used directly with the REST API. In this case, when searching on the date field, the REST API does not use date_from or date_to, but **from** and **to**. For completeness, you can also use the **date** field with the REST API. Note that the REST API will not return an error if you use date_from or date_to, it will ignore these search parameters.

In this case you also do not need to convert the returned date value to a human readable format, MISP does not return a UNIX timestamp, it returns the event date in a human readable (ISO 8601) format.

In [ ]:
print("Search for \033[92mevents after and before a specific date\033[90m")
result = requests.post("{}/events/restSearch".format(misp_url), headers=headers, verify=misp_verifycert,
                           json={"returnFormat":"json", "from":"2023-07-04", "to": "2023-07-06"})
table = eventstable(["Event ID", "Event", "Date", "Published"])
if not "errors" in result.json():    
    for event in result.json().get("response", []):
        table.add_row([event["Event"]["id"], event["Event"]["info"], event["Event"]["date"], event["Event"]["published"]])
else:
    print(result.text())
print(table.get_string(sortby="Date"), "\n\n")

## ES:4 What is the "First recorded change" of an event?

You probably noticed that when you view the details of MISP event, there's another useful timestamp, the *First recorded change*. Although this timestamp is displayed on the events detail page, it's actually calculated on the changes to **attributes** (or objects) (see [app/Controller/EventsController.php](https://github.com/MISP/MISP/blob/2.4/app/Controller/EventsController.php#L1469) ), more specifically on the first time an attribute (or object) was added to the MISP event. 

This timestamp is set automatically by MISP on the server where the event is created.

![helpers/timestamp_firstchange.png](helpers/timestamp_firstchange.png)

Because this timestamp is calculated on attribute (or object) information, you now need to set the PyMISP search controller to **attributes** (or objects) or use the attributes (or objects) REST API endpoint. In this playbook we first review how to do the search for attributes. You can apply the same approach when searching for objects.

# Attribute search context

## AS:1 Search for attributes on their last change timestamp

You can search for attributes based on the timestamp of their last change with the parameter **attribute_timestamp**. This [attribute timestamp](https://www.misp-standard.org/rfc/misp-standard-core.html#name-timestamp-2) is set automatically on the server where the attribute is last modified (or created).

The attribute_timestamp corresponds with the value displayed in the date column.

![helpers/timestamp_attribute_1.png](helpers/timestamp_attribute_1.png) 

### Search for actionable indicators only

For attributes it is also useful to consider if you want to include all attributes or only those which are marked as **actionable**. If you only want the attributes marked as actionable, then add the additional filter **to_ids** with a value of True. Typically if you search for data that needs to be send to security devices then you only include the actionable indicators.

### Use PyMISP to search for attributes on their last change timestamp

In this section, because we search for attributes, you have to set the PyMISP search controller to use **attributes** (instead of the default events). Then use the parameter **attribute_timestamp** to search for the last change timestamps. By default the information returned by the -attribute- search will not contain event information such as the even last change timestamp or event published timestamp. In some cases it's interesting to also receive this information. Set the parameter **include_context** to True to also include the event context.

In [ ]:
print("Search for \033[92mattributes changed after a specific timestamp\033[90m")
attributes = misp.search("attributes", attribute_timestamp="3d", pythonify=True)
table = eventstable(["Event ID", "Event", "Value", "Attribute timestamp", "Event last change", "Published date", "Published"])
for attribute in attributes:
    table.add_row([attribute.Event.id, attribute.Event.info, attribute.value, attribute.timestamp, 
                           attribute.Event.timestamp if hasattr(attribute.Event, "timestamp") else "Unknown", 
                           attribute.Event.publish_timestamp if hasattr(attribute.Event, "publish_timestamp") else "Unknown", attribute.Event.published])
print(table.get_string(sortby="Attribute timestamp"), "\n\n")

print("Search for \033[92mattributes changed after a specific timestamp\033[90m - with context")
# Added event context with include_context=True
attributes = misp.search("attributes", attribute_timestamp="3d", pythonify=True, include_context=True)
table = eventstable(["Event ID", "Event", "Value", "Attribute timestamp", "Event last change", "Published date", "Published"])
for attribute in attributes:
    table.add_row([attribute.Event.id, attribute.Event.info, attribute.value, attribute.timestamp, 
                           attribute.Event.timestamp if hasattr(attribute.Event, "timestamp") else "Unknown", 
                           attribute.Event.publish_timestamp if hasattr(attribute.Event, "publish_timestamp") else "Unknown", attribute.Event.published])
print(table.get_string(sortby="Attribute timestamp"), "\n\n")  

In [ ]:
print("Search for \033[92mattributes changed after a specific timestamp\033[90m and only return the \033[92mactionable\033[90m indicators")
attributes = misp.search("attributes", attribute_timestamp="3d", to_ids=True, pythonify=True)
table = eventstable(["Event ID", "Event", "Value", "Attribute timestamp"])
for attribute in attributes:
    table.add_row([attribute.Event.id, attribute.Event.info, attribute.value, attribute.timestamp])
print(table.get_string(sortby="Attribute timestamp"), "\n\n")

### Use REST API to search for attributes on their last change timestamp

The REST API is similar as the interactions with PyMISP. In this case you have to rely on the `attribute_timestamp` field in the JSON request: `{"returnFormat":"json", "attribute_timestamp": "yourdate"}`. Remember though, that in this case you have to use the **attributes** endpoint, and not the events endpoint.

In [ ]:
print("Search for \033[92mattributes changed after a specific timestamp\033[90m")
result = requests.post("{}/attributes/restSearch".format(misp_url), headers=headers, verify=misp_verifycert,
                           json={"returnFormat":"json", "attribute_timestamp":"3d"})
table = eventstable(["Event ID", "Event", "Value", "Attribute timestamp"])
if not "errors" in result.json():
    for attribute in result.json().get("response", []).get("Attribute", []):
        table.add_row([attribute["Event"]["id"], attribute["Event"]["info"], attribute["value"], unix_timestamp_to_date(attribute["timestamp"])])
print(table.get_string(sortby="Attribute timestamp"), "\n\n")

## AS:2 Search for attributes on their first seen or last seen value

You can also search for first seen and last seen values of attributes. The first seen and last seen are set manually by the threat author.

![helpers/timestamp_attribute_seen.png](helpers/timestamp_attribute_seen.png)

When searching for attributes with a last seen or first seen value it makes sense to also only focus on those attributes that have been marked as actionable, meaning the **to_ids** flag is set to True.

### Use PyMISP to search for attributes on their last seen or first seen value

The parameters that refer to the last seen or first seen value in the PyMISP search function are **last_seen** and **first_seen**. You can combine these queries, or only filter for one of the values.

In [ ]:
print("Search for attributes with a \033[92mfirst seen or last seen timestamp\033[90m")
attributes = misp.search("attributes", first_seen="15d", to_ids=True,  pythonify=True, include_context=True)
table = eventstable(["Event ID", "Event", "Value", "Attribute timestamp", "First seen"])
for attribute in attributes:
    table.add_row([attribute.Event.id, attribute.Event.info, attribute.value, attribute.timestamp, attribute.first_seen])
print(table.get_string(sortby="First seen"), "\n\n")

### Use REST API to search for attributes on their last seen or first seen value

It should not come as a surprise that the REST API also uses **last_seen** and **first_seen** as search criteria. Important to remark is that the date returned by the REST API is not in UNIX timestamp format, but in ISO 8601 format (hence also the reason why we don't convert it to a human readable string with `unix_timestamp_to_date`).

In [ ]:
print("Search for attributes with a \033[92mfirst seen or last seen timestamp\033[90m")
result = requests.post("{}/attributes/restSearch".format(misp_url), headers=headers, verify=misp_verifycert,
                           json={"returnFormat":"json", "first_seen":"10d", "to_ids":"1"})
table = eventstable(["Event ID", "Event", "Value", "Attribute timestamp", "Attribute first seen"])
if not "errors" in result.json():
    for attribute in result.json().get("response", []).get("Attribute", []):
        table.add_row([attribute["Event"]["id"], attribute["Event"]["info"], attribute["value"], unix_timestamp_to_date(attribute["timestamp"]), attribute["first_seen"]])
print(table.get_string(sortby="Attribute timestamp"), "\n\n")

## AS:3 Search for attributes in events with a specific recent change or published timestamp

So far we have searched for attributes and filtered on the timestamps associated with these attributes. MISP also allows you to search for attributes, and filter on the timestamps associated with the event in which these attributes are enclosed. Obviously you can also achieve this by searching for events and then working your way through the attributes (event per event), but in a lot of cases it's more convenient to search for the attributes, and use the event information  as an additional filter criteria.

### Use PyMISP to search for attributes in events with a specific recent change or published timestamp

In this case we have to set the controller to attributes, and then use **event_timestamp** as the filter to query for recently changed events. Likewise, if you want to filter for events with a recent published timestamp you can use the filter **publish_timestamp**. In this case we also include only the attributes that are actionable (to_ids is set to True).

In [ ]:
print("Search for \033[92mattributes\033[90m in events with a specific recent \033[92mtimestamp\033[90m change")
attributes = misp.search("attributes", event_timestamp="2d", to_ids=True, pythonify=True, include_context=True)
table = eventstable(["Event ID", "Event", "Event last change", "Value", "Attribute timestamp"])
for attribute in attributes:
    table.add_row([attribute.Event.id, attribute.Event.info, attribute.Event.timestamp, attribute.value, attribute.timestamp])
print(table.get_string(sortby="Attribute timestamp"), "\n\n")


print("Search for \033[92mattributes\033[90m in events with a specific recent \033[92mpublished timestamp\033[90m")
attributes = misp.search("attributes", publish_timestamp="2d", to_ids=True, pythonify=True, include_context=True)
table = eventstable(["Event ID", "Event", "Event publish timestamp", "Value", "Attribute timestamp"])
for attribute in attributes:
    table.add_row([attribute.Event.id, attribute.Event.info, attribute.Event.publish_timestamp, attribute.value, attribute.timestamp])
print(table.get_string(sortby="Attribute timestamp"), "\n\n")

### Use REST API to search for attributes in events with a specific recent change or published timestamp

The REST API is similar as the interactions with PyMISP, but don't forget to use the attributes endpoint. 

## AS:4 Search for attributes in events with a specific date field

Similarily as how you can search for events based on the **date** field you can also use this criteria to search for attributes part of events matching the **date_from** and / or **date_to** criteria.

### Use PyMISP and the REST API to search for attributes in events with a specific date field

In [ ]:
print("Search for \033[92mattributes\033[90m in events\033[92m after and before a specific date\033[90m")
attributes = misp.search("attributes", date_from="2023-07-06", date_to="2023-07-09", to_ids=True,  pythonify=True, include_context=True)
table = eventstable(["Event ID", "Event", "Event date", "Value", "Attribute timestamp"])
for attribute in attributes:
    table.add_row([attribute.Event.id, attribute.Event.info, attribute.Event.date, attribute.value, attribute.timestamp])
print(table.get_string(sortby="Attribute timestamp"), "\n\n") 

## AS:5 Search for first change of an attribute

We concluded the previous section on event searching with the *First recorded change* and this playbook has yet not provided an easy way to provide you this timestamp.

![helpers/timestamp_firstchange.png](helpers/timestamp_firstchange.png)


### Use PyMISP or REST API to search for the first change

Unfortunately there isn't an easy way to have PyMISP or the REST API return a sorted list of attributes. In order to get to the first recorded change you have to
- Initialise a variable on timestamp far ahead in time
- Get all the attributes for a specific event
    - We do not request them as Python objects, because we want only want to timestamp in *timestamp* format, not as a datetime object
- Loop through all the attributes and whenever the timestamp value is lower, update our variable
- In a second loop, attempt the above again, but now query for the objects in the event.
- This will eventually get use our *First recorded change* value based on attribute and object changes

In [ ]:
event_id_to_search = "2978"

print("Search for \033[92mfirst recorded change in an event\033[90m")
base_timestamp = 8551954799
timestamp = base_timestamp  # The time is Thu Dec 31 2240 22:59:59 GMT+0000. We're using MISP v6.2.92-ish by then
attributes = misp.search("attributes", eventid=event_id_to_search)
if len(attributes) > 0:
    for attribute in attributes.get("Attribute", []):
        if int(attribute["timestamp"]) < timestamp:
            timestamp = int(attribute["timestamp"])

objects = misp.search("objects", eventid=event_id_to_search)
if len(objects) > 0:
    for mispobject in objects:
        if int(mispobject["Object"]["timestamp"]) < timestamp:
            timestamp = int(mispobject["Object"]["timestamp"])
        
if timestamp < base_timestamp:     
    print("The first recorded change is: \033[92m{}\033[90m".format(unix_timestamp_to_date(timestamp)))
else:
    print("Unable to get the first recorded change.")
print("\n\n")

# Object search context

## OS:1 Search for objects

You already had a preview on how the PyMISP search function can be used to search for objects. It's as easy as telling the controller to use **objects**, and then use the same search criteria as you use for searching for attributes.

### Use PyMISP to search for objects based on their last timestamp change

A simple example demonstrate the use of PyMISP for searching of objects based on their last change timestamp with **attribute_timestamp**.

There are some peculiar aspects when working with Objects contrary to Attributes
- Contrary to with attributes, with `object.Event` you need to query for the dictionary value
- There is no `object_timestamp`, instead use `attribute_timestamp`

In [ ]:
print("Search for \033[92mobjects\033[90m with a recent \033[92mtimestamp change\033[90m")
objects = misp.search("objects", attribute_timestamp="10d", pythonify=True, include_context=True)
table = eventstable(["Event ID", "Event", "Object name", "Object UUID", "Comment", "Timestamp"])
for mispobject in objects:
    table.add_row([mispobject.Event["id"], mispobject.Event["info"], mispobject.name, mispobject.uuid, mispobject.comment, mispobject.timestamp])
print(table.get_string(sortby="Timestamp"), "\n\n") 

# Sightings

A [sighting](https://www.misp-standard.org/rfc/misp-standard-core.html#name-sighting) is an ascertainment which describes whether an attribute has been seen under a given set of conditions. The sighting can include the organisation who sighted the attribute or can be anonymised. 

## SI:1 Adding sightings

When you add sightings you can choose to add the sighting to one specific attribute in one event or add a *global* sighting. The latter means that the sighting is added to the attribute, regardless in which event the attribute is found. When you add sightings you can specify the **date_sighting**, when the referenced attribute is sighted. This date_sighting is expressed in seconds (decimal) elapsed since 1st of January 1970 (Unix timestamp).

### Use PyMISP to add sightings

In PyMISP you can use the function **add_sighting** and the supply a JSON describing the sighting. As you can learn from the example, you can supply the sighting date in a human readable format (ISO 8601) or as a UNIX timestamp.

In [ ]:
attribute_value = "167.89.16.17"
attribute_uuid = "5650a1bf-c25a-4e0a-a45a-f9dc10c0b6ff"

print("Adding sightings for \033[92m{}\033[90m\n\n".format(attribute_value))
date_human_readable = "2023-07-11"
date_unix_timestamp = 1689067634
print(misp.add_sighting({"value":"167.89.16.17", "attribute_uuid":attribute_uuid, "date_sighting":date_human_readable, "source":"MISP playbook - human readable date"}))
print(misp.add_sighting({"value":"167.89.16.17", "attribute_uuid":attribute_uuid, "date_sighting":date_unix_timestamp, "source":"MISP playbook - UNIX timestamp"}))

## SI:2 Search for sightings

Unfortunately in MISP it's not possible to search for a list of "recent" sightings. What's more, adding a sighting to an attribute does not change the last update timestamp of the event or the attribute. There are two ways though to search (or process) sightings, either query for them based on an event ID, or have them included in the results when you search for attributes.

### Use PyMISP to search for sightings

The **search_sightings** function of PyMISP allows you to search for sightings. You can filter the results for attributes, events or the type of sighting. What concerns timestamps you can filter the search results on sightings in events published after a specific date (with **publish_timestamp**) or for events that have their date set before (with **date_to**) or after (with **date_from**) a given date.

The returned timestamp is in UNIX timestamp format, that is why we transform it to a human readable format with `unix_timestamp_to_date`.

In [ ]:
event_id_to_search = 2655

print("Search for \033[92msightings\033[90m in a specific event")
result = misp.search_sightings(context="event", context_id=event_id_to_search, include_attribute=True, include_event_meta=True)
table = eventstable(["Attribute value", "Attribute ID", "Sighting source", "Sighting type", "Sighting timestamp"])
for sighting in result:
    table.add_row([sighting["Sighting"]["value"], sighting["Sighting"]["attribute_id"], sighting["Sighting"]["source"], sighting["Sighting"]["type"], unix_timestamp_to_date(sighting["Sighting"]["date_sighting"])])
print(table.get_string(sortby="Sighting timestamp"), "\n\n")

### Use the REST API to process sightings timestamps with the attributes or event endpoint

If you process attributes (or events) you can check for the presence of sightings by setting the parameter **includeSightings** to True. If there are sightings, then you can use the timestamp of the sighting as the reference to when the sighting was made. 

The returned timestamp is in UNIX timestamp format, that is why we transform it to a human readable format with `unix_timestamp_to_date`.

In [ ]:
event_id_to_search = "2655"


print("Search for attributes in a specific event and then check if there are \033[92msightings\033[90m")
result = requests.post("{}/attributes/restSearch".format(misp_url), headers=headers, verify=misp_verifycert,
                           json={"returnFormat":"json", "includeSightings":1, "includeContext":1, "eventid":event_id_to_search})
table = eventstable(["Event ID", "Event", "Attribute value", "Attribute timestamp", "Sighting source", "Sighting type", "Sighting timestamp"])
if not "errors" in result.json():    
    for attribute in result.json().get("response", []).get("Attribute", []):
        if len(attribute.get("Sighting", [])) > 0:
            for sighting in attribute["Sighting"]:
                table.add_row([attribute["Event"]["id"], attribute["Event"]["info"], attribute["value"], unix_timestamp_to_date(attribute["timestamp"]), sighting["source"], sighting["type"], unix_timestamp_to_date(sighting["date_sighting"])])
else:
    print(result.text())
print(table.get_string(sortby="Sighting timestamp"), "\n\n")

# Proposal / ShadowAttributes

Proposals or [ShadowAttributes](https://www.misp-standard.org/rfc/misp-standard-core.html#name-shadowattribute-2) are 3rd party created attributes that either propose to add new information to an event or modify existing information. They are not meant to be actionable until the event creator accepts them - at which point they will be converted into attributes or modify an existing attribute. They are similar in structure to Attributes but additionally carry a reference to the creator of the ShadowAttribute as well as a revocation flag.

## PS1: Search for proposals or shadowattributes

Unfortunately it's not possible to search for a list of "recent" proposals. What's more, a proposal to an event, does not change the last update timestamp of the event or the attribute.

### Use PyMISP to search for proposals

The PyMISP function **attribute_proposals** searches for proposals in a specific event.

In [ ]:
event_id_to_search = 2655

print(misp.attribute_proposals(event_id_to_search))

### Use the REST API to process proposal timestamps

If you process attributes (or events) you can check for the presence of proposals by setting the parameter **includeProposals** to True. If there are proposals, then you can use the timestamp of the proposal as the reference to when the proposal was made.

The returned timestamp is in UNIX timestamp format, that is why we transform it to a human readable format with `unix_timestamp_to_date`.

In [ ]:
event_id_to_search = "2655"


print("Search for attributes in a specific event and then check if there are \033[92mproposals\033[90m")
result = requests.post("{}/attributes/restSearch".format(misp_url), headers=headers, verify=misp_verifycert,
                           json={"returnFormat":"json", "includeProposals":1, "includeContext":1, "eventid":event_id_to_search})
table = eventstable(["Event ID", "Event", "Attribute value", "Attribute timestamp", "Proposal value", "Proposal timestamp"])
if not "errors" in result.json():    
    for attribute in result.json().get("response", []).get("Attribute", []):
        if len(attribute.get("ShadowAttribute", [])) > 0:
            for shadow in attribute["ShadowAttribute"]:
                table.add_row([attribute["Event"]["id"], attribute["Event"]["info"], attribute["value"], unix_timestamp_to_date(attribute["timestamp"]), shadow["value"], unix_timestamp_to_date(shadow["timestamp"])])
else:
    print(result.text())
print(table.get_string(sortby="Proposal timestamp"), "\n\n")

# Closure 
## EN:1 End of the playbook 

In [ ]:
print("\033[92m End of the playbook")


## External references 

- [The MISP Project](https://www.misp-project.org/)
- [MISP Cheat Sheet](https://www.misp-project.org/misp-training/cheatsheet.pdf)

## Technical details 

### Documentation

This playbook requires these Python **libraries** to exist in the environment where the playbook is executed. You can install them with `pip install <library>`.

```
PrettyTable
```

You need to have **network access** to 
- your MISP server (HTTP or HTTPS)

You need
- an **API key with MISP**
- - Under Global Actions, My Profile. Add an extra authentication key.
- - Add the API key (`misp_key`) and the MISP URL (`misp_url`) to `keys.py`
- - If you use a self-signed certificate set `misp_verifycert` to False